### Title: Mutation dependent synthetic lethyal pipeline
<font color='blue'> Author:</font>  Guangrong Qin<br/>

<font color='blue'> Contact:</font>  gqin@systemsbiology.org<br/>

<font color='blue'> Description:</font> This notebook is used to answer which gene knockout or gene knockdown  show sensitivity to certain gene mutation or the mutation of a group of genes. <br/>

<font color='blue'>Citations:</font> The functional screening data and omics data for cell lines is from the Depmap and CCLE project from the Broad institute (DepMap Public 20Q3). To use this jupyter notebook and the data which are used in the jupyter notebook, Please cite the following papers<br/>

Bahar Tercan, Guangrong Qin, Taek-Kyun Kim, Boris Aguilar, Christopher J. Kemp, Nyasha Chambwe, Ilya Shmulevich. SL-Cloud: A Computational Resource to Support Synthetic Lethal Interaction Discovery. BioRxiv 2021.09.18.459450; doi: https://doi.org/10.1101/2021.09.18.459450

For this DepMap release:
DepMap, Broad (2020): DepMap 20Q3 Public. figshare. Dataset doi:10.6084/m9.figshare.11791698.v2.

For CRISPR datasets:
Robin M. Meyers, Jordan G. Bryan, James M. McFarland, Barbara A. Weir, ... David E. Root, William C. Hahn, Aviad Tsherniak. Computational correction of copy number effect improves specificity of CRISPR-Cas9 essentiality screens in cancer cells. Nature Genetics 2017 October 49:1779–1784. doi:10.1038/ng.3984. PMID: 29083409

Dempster, J. M., Rossen, J., Kazachkova, M., Pan, J., Kugener, G., Root, D. E., & Tsherniak, A. (2019). Extracting Biological Insights from the Project Achilles Genome-Scale CRISPR Screens in Cancer Cell Lines. BioRxiv, 720243.

For omics datasets:
Mahmoud Ghandi, Franklin W. Huang, Judit Jané-Valbuena, Gregory V. Kryukov, ... Todd R. Golub, Levi A. Garraway & William R. Sellers. 2019. Next-generation characterization of the Cancer Cell Line Encyclopedia. Nature 569, 503–508 (2019).PMID: 31068700


In [ ]:
#Check the required libraries
try:
    from google.cloud import bigquery
    print("module 'google-cloud-bigquery' is installed")
except ModuleNotFoundError:
    !pip install google-cloud-bigquery
    from google.cloud import bigquery

try:
    import ipywidgets as widgets
    print("module 'ipywidgets' is installed")
except ModuleNotFoundError:
    !pip install ipywidgets
    import ipywidgets as widgets

try:
    import pyarrow
    print("module 'pyarrow' is installed")
except ModuleNotFoundError:
    !pip install pyarrow
    import pyarrow

try:
    import pandas as pd
    print("module 'pandas' is installed")
except ModuleNotFoundError:
    !pip install pandas
    import pandas as pd

try:
    import numpy as np
    print("module 'numpy' is installed")
except ModuleNotFoundError:
    !pip install numpy
    import numpy as np

try:
    from scipy import stats    
    print("module 'scipy' is installed")
except ModuleNotFoundError:
    !pip install scipy
    from scipy import stats    

try:
    import statsmodels.stats.multitest as multi   
    print("module 'statsmodels' is installed")
except ModuleNotFoundError:
    !pip install statsmodels
    import statsmodels.stats.multitest as multi
        
import sys
sys.path.append('../Scripts/')
import MDSLP

In [ ]:
# users need to run the following commend in their local machine or throught the notebook.
# Make sure to install the google cloud in the local envirionment. For more detail of gcloud installation, please see support from https://cloud.google.com/sdk/docs/install

!gcloud auth application-default login

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
# Users need to a google cloud project to query the data in the BigQuery tables. 
project_id='syntheticlethality' #users need to have their own google project to query the datasets on ISB-CGC
client = bigquery.Client(project_id)

#### Get mutation data from CCLE, CRISPR gene knockout effects from Depmap and shRNA gene knockdown gene dependency data from demeter2 v6. Depmap version 20Q3 is used for the following analysis

In [ ]:
#This step may take a little bit longer time
Mut_mat = MDSLP.get_ccle_mutation_data(project_id) # Get mutation table for the ccle cell lines (version: Depmap 20Q3)
Demeter_data = MDSLP.get_demeter_shRNA_data(project_id) # Get shRNA-based gene knockdown effects from the Depmap project (Demeter2)
Depmap_matrix = MDSLP.get_depmap_crispr_data(project_id) #Get the CRISPR-based gene knockout effects from the Depmap project (version: Depmap 20Q3) 

##### 
You are expecting to see the message above as follows:
Unnamed: 0 <br/>
AZ521_STOMACH<br/>
GISTT1_GASTROINTESTINAL_TRACT<br/>
MB157_BREAST<br/>
SW527_BREAST<br/>
<br/>
It means these cell lines are not included for the analysis as they show mismatching annotation from different datasets.


#### Set user input:
###### 1, Data_source: only two options are avaiable, "shRNA" or "Crispr", datatype: string
###### 2, Mutated genes to be interested. It can be a list of genes or one single gene in a list format. 
###### 3, Tumor types to be included in the analysis. Users can select 'pancancer' or select one or multiple tumor types to theirs interests.

In [ ]:
# User input; The natural language question we ask here is which gene show senthetic lethality with the gene being mutated.
Data_source = "shRNA" # only two options are avaiable, "shRNA" or "Crispr", datatype: string
Gene_list = ['BRCA2'] # data type: list of gene symbols


In [ ]:
# ID mapping between the CCLE annotation and input gene symbols
id_mapping, Gene_list_matched = MDSLP.GeneSymbol_standardization(Gene_list, project_id)


#### Select cell lines in specific tumor types into analysis.

In [ ]:
query = ''' 
SELECT DepMap_ID, primary_disease,TCGA_subtype
FROM `syntheticlethality.DepMap_public_20Q3.sample_info_Depmap_withTCGA_labels` 
'''
sample_info = client.query(query).result().to_dataframe()

pancancer_cls = sample_info.loc[~sample_info['primary_disease'].isin(['Non-Cancerous','Unknown','Engineered','Immortalized'])]
pancancer_cls = pancancer_cls.loc[~(pancancer_cls['primary_disease'].isna())]

TCGA_list = [x for x in list(set(pancancer_cls['primary_disease'])) if x == x]

Not_none_values = filter(None.__ne__, TCGA_list)
TCGA_list = list(Not_none_values)

tumor_type = widgets.SelectMultiple(
    options=['pancancer'] + TCGA_list  ,
    value=[],
    description='Tumor type',
    disabled=False
)
display(tumor_type)

#### Select shRNA dataset or Crispr dataset to infer synthetic lethality pairs for mutated genes! 

In [ ]:
Data_source = "shRNA"
if Data_source == "shRNA":
    result_shRNA = MDSLP.Mutational_based_SL_pipeline(list(tumor_type.value), Gene_list_matched, Mut_mat, Demeter_data, Data_source,project_id)
    if result_shRNA.shape[0] > 0:
        result_shRNA_sig = result_shRNA.loc[result_shRNA['FDR_all_exp'] < 0.05]
        result_shRNA_sig = result_shRNA_sig.loc[result_shRNA_sig['ES']<0] # ES < 0 represents SL pairs

In [ ]:
result_shRNA_sig.sort_values(by = ['FDR_all_exp'])

In [ ]:
result_shRNA_sig.to_csv("BRCA2_shRNA_sig.csv")

In [ ]:
Data_source = "Crispr"
if Data_source == "Crispr":
    result_Crispr = MDSLP.Mutational_based_SL_pipeline(list(tumor_type.value), Gene_list_matched, Mut_mat, Depmap_matrix, Data_source,project_id)
    if result_Crispr.shape[0] > 0:
        result_Crispr_sig = result_Crispr.loc[result_Crispr['FDR_all_exp'] < 0.05]
        result_Crispr_sig = result_Crispr_sig.loc[result_Crispr_sig['ES'] < 0]  # ES < 0 represents SL pairs
    else:
        result_Crispr_sig = pd.DataFrame()

In [ ]:
result_Crispr_sig.sort_values(by = ['FDR_all_exp'])

In [ ]:
result_Crispr_sig.to_csv("BRCA2_Crispr_sig.csv")

###### Result interpretation 
result_Crispr_sig or result_shRNA_sig contains the synthetic lethal gene pairs predicted from this pipeline.<br/>
###### table annotation:
Gene_mut: mutated genes;<br/>
Gene_kd: gene knockdown or knock out <br/>
Mutated_samples: Number of mutated cell lines in the selected tumor type<br/>
pvalue: p-value result from t-test<br/>
ES: effect size of gene effects between the mutated group and wild type group<br/>
FDR_all_exp: FDR for p-value for all analysis<br/>
FDR_by_gene: FDR for p-value by one gene mutation<br/>
Tumor_type: tumor types in analysis